In [2]:
import numpy as np
import os
import glob
import pandas as pd
import PIL
import PIL.Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [3]:
# Set the seed for reproducibility
tf.random.set_seed(42)

In [4]:
batch_size = 32
img_height = 192
img_width = 192

We have replaced the custom model architecture with a pre-trained model, such as MobileNetV2, which is a popular choice for transfer learning due to its efficiency and effectiveness. 

In [5]:
# Load the MobileNetV2 pre-trained model for transfer learning
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False  # Freeze the pre-trained weights

In [6]:
# Add a custom head to the model for facial expression classification
num_classes = 7
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes)
])

In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [8]:
# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_192 (Func  (None, 6, 6, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 7)                 903       
                                                                 
Total params: 2422855 (9.24 MB)
Trainable params: 164871

In [9]:
final_data_path = "my_drive/classes/"

In [10]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  final_data_path,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
class_names = train_ds.class_names
print(class_names)

Found 74532 files belonging to 7 classes.
Using 59626 files for training.
['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [11]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  final_data_path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 74532 files belonging to 7 classes.
Using 14906 files for validation.


In [12]:
# Normalize the data
normalization_layer = tf.keras.layers.Rescaling(1/255)
train_ds_normalized = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds_normalized = val_ds.map(lambda x, y: (normalization_layer(x), y))

In [13]:
# Cache and prefetch the datasets
AUTOTUNE = tf.data.AUTOTUNE
train_ds_normalized = train_ds_normalized.cache().prefetch(buffer_size=AUTOTUNE)
val_ds_normalized = val_ds_normalized.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
# Train the model and plot the training and validation curves
history = model.fit(train_ds_normalized,
                    validation_data=val_ds_normalized,
                    epochs=5)  # You can increase the number of epochs as needed


In [ ]:
# Plot the training and validation curves
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()